In [3]:
import aocd
import dataclasses
import numpy as np
import enum

real_data = aocd.get_data(day=18, year=2022)
test_data = """2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5"""

In [278]:
from typing import Sequence, Union, Tuple, Type
import copy
import itertools
import sys
import random

@dataclasses.dataclass
class Cube:
    """Defines a cube instance.
    
    Args:
        pos: the 3d position.
    """
    pos: Tuple[int, int, int]

    def scan_neighbours(self, coords: np.ndarray):
        """Tells each cubes how many open surfaces there are."""
        total = np.sum(np.sqrt(np.sum((self.pos - coords) ** 2, axis=1)) == 1)
        self.n_open = 6 - total
            
@dataclasses.dataclass
class SolverA:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.lines = self.raw_data.splitlines()
        num_lines = []
        cubes = []
        for line in self.lines:
            position = [int(x) for x in line.split(",")]
            num_lines.append(position)
            cubes.append(Cube(pos=tuple(position)))
        self.coords = np.array(num_lines)
        self.cubes = cubes
        


    def answer(self) -> int:
        for cube in self.cubes:
            cube.scan_neighbours(self.coords)
        return np.sum([cube.n_open for cube in self.cubes])

In [279]:
SolverA(test_data).answer()

64

In [281]:
SolverA(real_data).answer()

4636

In [295]:
from typing import Sequence, Union, Tuple, Type
import copy
import itertools
import sys
import random

# Let's find the congruent blocks from the outside and the 
# answer should be the total surface area of that block minus 
# the outtermost cube surface.


@dataclasses.dataclass
class Cube:
    """Defines a cube instance.
    
    Args:
        pos: the 3d position.
    """
    pos: Tuple[int, int, int]

    def scan_neighbours(self, coords: np.ndarray):
        """Tells each cubes how many open surfaces there are."""
        total = np.sum(np.sqrt(np.sum((self.pos - coords) ** 2, axis=1)) == 1)
        self.n_open = 6 - total

def get_cubes_coords(cubes: Sequence[Cube]) -> np.ndarray:
    """Finds the coordinates of all the cubes in the list."""
    coords = []
    for cube in cubes:
        coords.append(cube.pos)
    return np.array(coords)
            
@dataclasses.dataclass
class SolverB:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.lines = self.raw_data.splitlines()

        num_lines = []
        for line in self.lines:
            num_lines.append([
                int(x) for x in line.split(",")
            ])
        self.coords = np.array(num_lines)
        
        v_max = np.amax(self.coords)
        v_min = np.amax(np.min(self.coords))
        v_range = int(v_max - v_min + 3)
        self.canvas = np.zeros([v_range, v_range, v_range])
        self.imax = v_range - 1

        # load the data onto the canvas
        c = 0
        for coord in self.coords:
            coord = tuple(coord - v_min + 1)
            self.canvas[coord] = 1

    def get_air_blocks(self) -> Sequence[Tuple[int, int, int]]:
        """Finds the congruent block of air surrounding the lava.
        
        Returns:
            a list of all the coordinates of the air blocks.
        """
        air_blocks = []
        evaluated = []
        queue = [(0, 0, 0)]
        expansions = [
            [0, 0, 1],
            [0, 0, -1],
            [0, 1, 0],
            [0, -1, 0],
            [1, 0, 0],
            [-1, 0, 0]
        ]
        while len(queue) > 0:
            position = queue.pop(0)
            evaluated.append(position)

            if self.canvas[position] == 0:
                air_blocks.append(position)
                for expansion in expansions:
                    t = tuple(np.clip([a + b for a, b in zip(expansion, position)], 0, self.imax))
                    if (t in queue) or (t in evaluated):
                        continue
                    queue.append(t)

        return air_blocks
    
    def answer(self) -> int:
        air_blocks = self.get_air_blocks()
        cubes = []
        for air_block_pos in air_blocks:
            cubes.append(Cube(air_block_pos))
            
        air_cube_coords = get_cubes_coords(cubes)
        for cube in cubes:
            cube.scan_neighbours(air_cube_coords)
            
        exterior_surface = 6 * (self.imax + 1) ** 2
        return np.sum([cube.n_open for cube in cubes]) - exterior_surface

In [296]:
SolverB(test_data).answer()

58

In [298]:
SolverB(real_data).answer()

2572